# Extração dos valores liberados pelo FNDE

Importando os pacotes necessários

In [1]:
import urllib.request

In [2]:
import re

In [3]:
import pandas as pd

In [4]:
import ssl

Desabilitando a verificação do contexto

In [5]:
context = ssl._create_unverified_context()

Criando a lista para armazenar os anos da pesquisa

In [6]:
anos = []

Definindo o ano inicial da pesquisa

In [7]:
primeiro_ano = 2018

Definindo o ano final da pesquisa

In [8]:
ultimo_ano = 2018

Carregando a lista com os anos a serem pesquisados

In [9]:
for ano in range(primeiro_ano, ultimo_ano + 1):
    anos.append(ano)

Separando os códigos do banco de dados

In [10]:
# ,'02', '0A', '0B', 'A3', 'Z9'
codigos = ['C7']

Definindo o padrão da string da url da pesquisa

In [11]:
padrao_url = 'https://www.fnde.gov.br/sigefweb/index.php/liberacoes?ano={}&programa={}&cnpj=&estado=0&tipoEntidade=todos&confirmar=Confirmar'

Criando uma lista para armazenar as urls da pesquisa

In [12]:
urls = []

Carregando a lista com as urls a serem pesquisadas

In [13]:
for ano in reversed(anos):
    for codigo in codigos:
        urls.append(padrao_url.format(ano, codigo))

Conferindo as urls carregadas na lista

In [14]:
len(urls)

1

In [15]:
urls

['https://www.fnde.gov.br/sigefweb/index.php/liberacoes?ano=2018&programa=C7&cnpj=&estado=0&tipoEntidade=todos&confirmar=Confirmar']

Criando uma lista para carregar as esferas pesquisadas em cada url

In [16]:
lista_esferas = []

Carregando a lista de esferas a partir das urls

In [17]:
for url in urls:
    with urllib.request.urlopen(url, context=context) as response:
       html = response.read().decode('ansi')
    esferas = re.findall(r'<tr><td><a href=\"(.*?)\">MUNICIPAL</a>', html)
    lista_esferas = lista_esferas + esferas

Conferindo a lista de esferas carregada

In [18]:
len(lista_esferas)

1

In [19]:
lista_esferas

['/sigefweb/index.php/liberacoes/resultado-parcial/ano/2018/programa/C7/tipoEntidade/todos/esfera/municipal']

Definindo a url do site do FNDE

In [20]:
site_fnde = 'https://www.fnde.gov.br'

Criando as listas das urls das entidades de cada programa e das UF de cada entidade

In [21]:
lista_entidades = []

Carregando a lista de entidades de cada programa

In [22]:
for esfera in lista_esferas:
    with urllib.request.urlopen(site_fnde + esfera, context=context) as response:
       html = response.read().decode('ansi')
    entidades = re.findall(r'<tr><td><a href=\"(.*?)\">', html)
    lista_entidades = lista_entidades + entidades

Conferindo o tamanho da lista de entidades

In [23]:
len(lista_entidades)

5512

Criando as listas onde serão armazenadas as informações das entidades pesquisadas

In [24]:
lista_totais = []

In [25]:
lista_dados = []

Carregando as listas com as informações de cada entidade. As expressões regulares foram construídas com a ferramenta disponível no site: http://www.regexlab.com/wild2regex e https://pythex.org/

In [ ]:
for entidade in lista_entidades:
    with urllib.request.urlopen(site_fnde + entidade, context=context) as response:
       html = response.read().decode('ansi')
    totais = re.findall(r'<tfoot><tr><td>&nbsp;</td><td>Total: </td><td>(.*?)</td>', html)
    lista_totais = lista_totais + totais
    dados = re.findall(r'<td>(.*?)</td>', html)
    lista_dados = lista_dados + dados

In [ ]:
lista_cnpjs = []

In [ ]:
lista_ufs = []

In [ ]:
lista_municipios = []

In [ ]:
for i, dado in enumerate(lista_dados):
    if i % 4 == 0:
        lista_cnpjs.append(dado)
    if i % 4 == 2:
        lista_ufs.append(dado)
    if i % 4 == 3:
        lista_municipios.append(dado)

In [ ]:
entidades_dict = {'UF': lista_ufs, 'MUNICIPIO': lista_municipios, 'CNPJ': lista_cnpjs, 'ALIMENTACAO': lista_totais}

In [ ]:
entidades_uf = pd.DataFrame.from_dict(entidades_dict)

Conferindo o conteúdo das listas das informações

In [ ]:
len(lista_ufs)

In [ ]:
len(lista_cnpjs)

In [ ]:
len(lista_bancos)

In [ ]:
len(lista_agencias)

In [ ]:
len(lista_contas)

In [ ]:
len(lista_programas)

In [ ]:
len(lista_entidades_problema)

In [ ]:
len(lista_ufs_entidades_problema)

Criando um dicionário com as informações pesquisadas

In [ ]:
entidades = {'UF': lista_ufs, 'CNPJ': lista_cnpjs, 'BANCO': lista_bancos, 'AGENCIA': lista_agencias, 'CONTA': lista_contas, 'PROGRAMA': lista_programas}

In [ ]:
entidades_problema = {'UF': lista_ufs_entidades_problema, 'ENTIDADE': lista_entidades_problema}

Criando um novo banco de dados a partir do dicionário entidades

In [ ]:
banco_entidades = pd.DataFrame.from_dict(entidades)

In [ ]:
banco_entidades_problema = pd.DataFrame.from_dict(entidades_problema)

In [ ]:
banco_entidades.head()

In [ ]:
banco_entidades_problema.head()

In [ ]:
banco_entidades = banco_entidades.drop_duplicates()

In [ ]:
banco_entidades_problema = banco_entidades_problema.drop_duplicates()

In [ ]:
banco_entidades.to_excel('municipais.xlsx', index=False)

In [ ]:
banco_entidades_problema.to_excel('municipais_problema.xlsx', index=False)

In [ ]:
contas = banco_entidades.merge(lista_programas, how='left', left_on='PROGRAMA', right_on='Código')

In [ ]:
contas.drop(columns=['PROGRAMA'])

In [ ]:
contas['PROGRAMA']

In [ ]:
entidades_problema